In [ ]:
%matplotlib inline
import numpy as np
import warnings
import matplotlib.pyplot as plt
import pandas as pd
warnings.filterwarnings('ignore')
np.random.seed(123)

In [ ]:
train=pd.read_excel("osdg.xlsx").drop(columns=["Unnamed: 0","doi","text_id","labels_negative","labels_positive","agreement"])
train

,text,sdg
0,"""""From a gender perspective, Paulgaard points ...",5
1,The average figure also masks large difference...,3
2,The Israel Oceanographic and Limnological Rese...,6
3,Previous chapters have discussed ways to make ...,2
4,The “War on Terror” and the Framework of Inter...,16
...,...,...
22091,"In consequence, the number of urban dwellers p...",3
22092,"Unlike the right to food and housing, the righ...",16
22093,"It also features individual accountability, wh...",4
22094,Groundwater quality can also be affected by co...,6


In [ ]:
#convert the label format to multi-label format
train[["sdg"]]=train[["sdg"]].values.astype(str)
one_hot=["sdg"] 
sdg_coded=pd.get_dummies(train[one_hot])
for i in sdg_coded.columns.values.tolist():
    train[i]=sdg_coded[i]
train=train.drop(columns = one_hot, axis = 1)
train

,text,sdg_1,sdg_10,sdg_11,sdg_12,sdg_13,sdg_14,sdg_15,sdg_16,sdg_2,sdg_3,sdg_4,sdg_5,sdg_6,sdg_7,sdg_8,sdg_9
0,"""""From a gender perspective, Paulgaard points ...",0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,The average figure also masks large difference...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,The Israel Oceanographic and Limnological Rese...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,Previous chapters have discussed ways to make ...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,The “War on Terror” and the Framework of Inter...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22091,"In consequence, the number of urban dwellers p...",0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
22092,"Unlike the right to food and housing, the righ...",0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
22093,"It also features individual accountability, wh...",0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
22094,Groundwater quality can also be affected by co...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.common.utils.utils import setup_outputdir
from autogluon.core.utils.loaders import load_pkl
from autogluon.core.utils.savers import save_pkl
import os.path
class MultilabelPredictor():
    """ Tabular Predictor for predicting multiple columns in table.
        Creates multiple TabularPredictor objects which you can also use individually.
        You can access the TabularPredictor for a particular label via: `multilabel_predictor.get_predictor(label_i)`

        Parameters
        ----------
        labels : List[str]
            The ith element of this list is the column (i.e. `label`) predicted by the ith TabularPredictor stored in this object.
        path : str
            Path to directory where models and intermediate outputs should be saved.
            If unspecified, a time-stamped folder called "AutogluonModels/ag-[TIMESTAMP]" will be created in the working directory to store all models.
            Note: To call `fit()` twice and save all results of each fit, you must specify different `path` locations or don't specify `path` at all.
            Otherwise files from first `fit()` will be overwritten by second `fit()`.
            Caution: when predicting many labels, this directory may grow large as it needs to store many TabularPredictors.
        problem_types : List[str]
            The ith element is the `problem_type` for the ith TabularPredictor stored in this object.
        eval_metrics : List[str]
            The ith element is the `eval_metric` for the ith TabularPredictor stored in this object.
        consider_labels_correlation : bool
            Whether the predictions of multiple labels should account for label correlations or predict each label independently of the others.
            If True, the ordering of `labels` may affect resulting accuracy as each label is predicted conditional on the previous labels appearing earlier in this list (i.e. in an auto-regressive fashion).
            Set to False if during inference you may want to individually use just the ith TabularPredictor without predicting all the other labels.
        kwargs :
            Arguments passed into the initialization of each TabularPredictor.

    """

    multi_predictor_file = 'multilabel_predictor.pkl'

    def __init__(self, labels, path, problem_types=None, eval_metrics=None, consider_labels_correlation=True, **kwargs):
        if len(labels) < 2:
            raise ValueError("MultilabelPredictor is only intended for predicting MULTIPLE labels (columns), use TabularPredictor for predicting one label (column).")
        self.path = setup_outputdir(path, warn_if_exist=False)
        self.labels = labels
        self.consider_labels_correlation = consider_labels_correlation
        self.predictors = {}  # key = label, value = TabularPredictor or str path to the TabularPredictor for this label
        if eval_metrics is None:
            self.eval_metrics = {}
        else:
            self.eval_metrics = {labels[i] : eval_metrics[i] for i in range(len(labels))}
        problem_type = None
        eval_metric = None
        for i in range(len(labels)):
            label = labels[i]
            path_i = self.path + "Predictor_" + label
            if problem_types is not None:
                problem_type = problem_types[i]
            if eval_metrics is not None:
                eval_metric = self.eval_metrics[label]
            self.predictors[label] = TabularPredictor(label=label, problem_type=problem_type, eval_metric=eval_metric, path=path_i, **kwargs)

    def fit(self, train_data, tuning_data=None, **kwargs):
        """ Fits a separate TabularPredictor to predict each of the labels.

            Parameters
            ----------
            train_data, tuning_data : str or autogluon.tabular.TabularDataset or pd.DataFrame
                See documentation for `TabularPredictor.fit()`.
            kwargs :
                Arguments passed into the `fit()` call for each TabularPredictor.
        """
        if isinstance(train_data, str):
            train_data = TabularDataset(train_data)
        if tuning_data is not None and isinstance(tuning_data, str):
            tuning_data = TabularDataset(tuning_data)
        train_data_og = train_data.copy()
        if tuning_data is not None:
            tuning_data_og = tuning_data.copy()
        else:
            tuning_data_og = None
        save_metrics = len(self.eval_metrics) == 0
        for i in range(len(self.labels)):
            label = self.labels[i]
            predictor = self.get_predictor(label)
            if not self.consider_labels_correlation:
                labels_to_drop = [l for l in self.labels if l != label]
            else:
                labels_to_drop = [self.labels[j] for j in range(i+1, len(self.labels))]
            train_data = train_data_og.drop(labels_to_drop, axis=1)
            if tuning_data is not None:
                tuning_data = tuning_data_og.drop(labels_to_drop, axis=1)
            print(f"Fitting TabularPredictor for label: {label} ...")
            predictor.fit(train_data=train_data, tuning_data=tuning_data, **kwargs)
            self.predictors[label] = predictor.path
            if save_metrics:
                self.eval_metrics[label] = predictor.eval_metric
            copy_trained(label)
        self.save()

    def predict(self, data, **kwargs):
        """ Returns DataFrame with label columns containing predictions for each label.

            Parameters
            ----------
            data : str or autogluon.tabular.TabularDataset or pd.DataFrame
                Data to make predictions for. If label columns are present in this data, they will be ignored. See documentation for `TabularPredictor.predict()`.
            kwargs :
                Arguments passed into the predict() call for each TabularPredictor.
        """
        return self._predict(data, as_proba=False, **kwargs)

    def predict_proba(self, data, **kwargs):
        """ Returns dict where each key is a label and the corresponding value is the `predict_proba()` output for just that label.

            Parameters
            ----------
            data : str or autogluon.tabular.TabularDataset or pd.DataFrame
                Data to make predictions for. See documentation for `TabularPredictor.predict()` and `TabularPredictor.predict_proba()`.
            kwargs :
                Arguments passed into the `predict_proba()` call for each TabularPredictor (also passed into a `predict()` call).
        """
        return self._predict(data, as_proba=True, **kwargs)

    def evaluate(self, data, **kwargs):
        """ Returns dict where each key is a label and the corresponding value is the `evaluate()` output for just that label.

            Parameters
            ----------
            data : str or autogluon.tabular.TabularDataset or pd.DataFrame
                Data to evalate predictions of all labels for, must contain all labels as columns. See documentation for `TabularPredictor.evaluate()`.
            kwargs :
                Arguments passed into the `evaluate()` call for each TabularPredictor (also passed into the `predict()` call).
        """
        data = self._get_data(data)
        eval_dict = {}
        for label in self.labels:
            print(f"Evaluating TabularPredictor for label: {label} ...")
            predictor = self.get_predictor(label)
            eval_dict[label] = predictor.evaluate(data, **kwargs)
            if self.consider_labels_correlation:
                data[label] = predictor.predict(data, **kwargs)
        return eval_dict

    def save(self):
        """ Save MultilabelPredictor to disk. """
        for label in self.labels:
            if not isinstance(self.predictors[label], str):
                self.predictors[label] = self.predictors[label].path
        save_pkl.save(path=self.path+self.multi_predictor_file, object=self)
        try:
          save_pkl.save(path="/content/drive/MyDrive/OSDG16/model/"+self.multi_predictor_file, object=self)
        except:
          print("failed when save to /content/drive/MyDrive/OSDG16/model file.")
          print("self.path：",self.path)
        print(f"MultilabelPredictor saved to disk. Load with: MultilabelPredictor.load('{self.path}')")

    @classmethod
    def load(cls, path):
        """ Load MultilabelPredictor from disk `path` previously specified when creating this MultilabelPredictor. """
        path = os.path.expanduser(path)
        if path[-1] != os.path.sep:
            path = path + os.path.sep
        return load_pkl.load(path=path+cls.multi_predictor_file)

    def get_predictor(self, label):
        """ Returns TabularPredictor which is used to predict this label. """
        predictor = self.predictors[label]
        if isinstance(predictor, str):
            return TabularPredictor.load(path=predictor)
        return predictor

    def _get_data(self, data):
        if isinstance(data, str):
            return TabularDataset(data)
        return data.copy()

    def _predict(self, data, as_proba=False, **kwargs):
        data = self._get_data(data)
        if as_proba:
            predproba_dict = {}
        for label in self.labels:
            print(f"Predicting with TabularPredictor for label: {label} ...")
            predictor = self.get_predictor(label)
            if as_proba:
                predproba_dict[label] = predictor.predict_proba(data, as_multiclass=True, **kwargs)
            data[label] = predictor.predict(data, **kwargs)
        if not as_proba:
            return data[self.labels]
        else:
            return predproba_dict

In [ ]:
labels = ["sdg_1","sdg_2","sdg_3","sdg_4","sdg_5","sdg_6","sdg_7","sdg_8","sdg_9","sdg_10","sdg_11","sdg_12","sdg_13","sdg_14","sdg_15","sdg_16"]  # which columns to predict based on the others
problem_types = ['binary' for i in range (16)]  # type of each prediction problem
save_path = './mutimodal_res'  # specifies folder to store trained models
eval_metrics_ = ['f1' for i in range (16)]

In [ ]:
multi_predictor = MultilabelPredictor(labels=labels, problem_types=problem_types, path=save_path,eval_metrics=eval_metrics_,consider_labels_correlation=False,sample_weight='auto_weight')
multi_predictor.fit(train, hyperparameters='multimodal',time_limit=None,presets=['best_quality'],num_gpus=1,excluded_model_types=['KNN', 'NN_MXNET', 'NN_TORCH'],num_stack_levels=1,num_bag_folds=8)

Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "./mutimodal_res/Predictor_sdg_15/"
AutoGluon Version:  0.6.0
Python Version:     3.8.10
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Sat Dec 10 16:00:40 UTC 2022
Train Data Rows:    22096
Train Data Columns: 1
Label Column: sdg_15
Preprocessing data ...
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11908.28 MB
	Train Data (Original)  Memory Usage: 20.16 MB (0.2% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.


Fitting TabularPredictor for label: sdg_15 ...


	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
			Fitting RenameFeatureGenerator...
		Fitting CategoryFeatureGenerator...
			Fitting CategoryMemoryMinimizeFeatureGenerator...
		Fitting TextSpecialFeatureGenerator...
			Fitting BinnedFeatureGenerator...
			Fitting DropDuplicatesFeatureGenerator...
		Fitting TextNgramFeatureGenerator...
			Fitting CountVectorizer for text features: ['text']
			CountVectorizer fit with vocabulary size = 10000
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('object', ['text']) : 1 | ['text']
	Types of features in processed data (raw dtype, special dtypes):
		('int', ['binned', 'text_special']) :    22 | ['text.char_count', 'text.word_count', 'text.capital_ratio', 'text.lower_ratio', 'text.digit_ratio', ...]
		('int', ['text_ngram'])             : 10001 

Fitting TabularPredictor for label: sdg_16 ...


	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
			Fitting RenameFeatureGenerator...
		Fitting CategoryFeatureGenerator...
			Fitting CategoryMemoryMinimizeFeatureGenerator...
		Fitting TextSpecialFeatureGenerator...
			Fitting BinnedFeatureGenerator...
			Fitting DropDuplicatesFeatureGenerator...
		Fitting TextNgramFeatureGenerator...
			Fitting CountVectorizer for text features: ['text']
			CountVectorizer fit with vocabulary size = 10000
		Reducing Vectorizer vocab size from 10000 to 1317 to avoid OOM error
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('object', ['text']) : 1 | ['text']
	Types of features in processed data (raw dtype, special dtypes):
		('int', ['binned', 'text_special']) :   22 | ['text.char_count', 'text.word_count', 'text.capital_ratio', 'text.lower_ratio', '

MultilabelPredictor saved to disk. Load with: MultilabelPredictor.load('./mutimodal_res/')


In [ ]:
test=pd.read_excel("english_library_test.xlsx")
test["text"]=test["title"]+' '+test['descriptions']
test=test.drop(columns=["title",'descriptions'])
test

,text
0,Do No Harm: The Rise of New Humanitarianism H...
1,Money vs. Barter History and Concepts This mod...
2,21st Century Perspectives on Macroeconomics In...
3,A Global Mitigation Scenario The three pillars...
4,A Job for the Future: The Innovation Broker Th...
...,...
895,Why Invest in Transboundary Basin Activities? ...
896,Winners and losers in the global economy In th...
897,Work in a Post Manufacturing World This video ...
898,Youth and Food: New Entrepreneurs in the Food ...


In [ ]:
predictions=pd.DataFrame(columns=["sdg_1","sdg_2","sdg_3","sdg_4","sdg_5","sdg_6","sdg_7","sdg_8","sdg_9","sdg_10","sdg_11","sdg_12","sdg_13","sdg_14","sdg_15","sdg_16"])

for i in range(1,17):
  predictor = TabularPredictor.load("/content/drive/MyDrive/OSDG16/V2model/Predictor_sdg_"+str(i))
  #res=predictor.predict(test)
  res=predictor.predict_proba(test)
  #predictions["sdg_"+str(i)]=res
  predictions["sdg_"+str(i)]=res[1]
  print(str(i)+" success!")
  predictions.to_excel("proba_V2English_Library_prediction.xlsx")

INFO:automm:Load pretrained checkpoint: /content/drive/MyDrive/OSDG16/V2model/Predictor_sdg_1/models/TextPredictor_BAG_L1/S1F1/text_nn/model.ckpt


Predicting: 0it [00:00, ?it/s]

INFO:automm:Load pretrained checkpoint: /content/drive/MyDrive/OSDG16/V2model/Predictor_sdg_1/models/TextPredictor_BAG_L1/S1F2/text_nn/model.ckpt


Predicting: 0it [00:00, ?it/s]

INFO:automm:Load pretrained checkpoint: /content/drive/MyDrive/OSDG16/V2model/Predictor_sdg_1/models/TextPredictor_BAG_L1/S1F3/text_nn/model.ckpt


...

INFO:automm:Load pretrained checkpoint: /content/drive/MyDrive/OSDG16/V2model/Predictor_sdg_16/models/TextPredictor_BAG_L1/S1F7/text_nn/model.ckpt


Predicting: 0it [00:00, ?it/s]

INFO:automm:Load pretrained checkpoint: /content/drive/MyDrive/OSDG16/V2model/Predictor_sdg_16/models/TextPredictor_BAG_L1/S1F8/text_nn/model.ckpt


Predicting: 0it [00:00, ?it/s]

16 success!


In [ ]:
for i in range(1,17):
  print("Classfier: "+str(i))
  predictor = TabularPredictor.load("/content/drive/MyDrive/OSDG16/V2model/Predictor_sdg_"+str(i))
  predictor.leaderboard()
  print("================================================")

Classfier: 1
                   model  score_val  pred_time_val      fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0    WeightedEnsemble_L3   0.856310      83.993402  18477.391385                0.062459           8.531392            3       True         14
1         XGBoost_BAG_L2   0.855953      80.867601  18126.866040                4.454695         179.862311            2       True         12
2   LightGBMLarge_BAG_L2   0.853782      80.287874  18466.761883                3.874969         519.758154            2       True         13
3    WeightedEnsemble_L2   0.853669      71.698163  17092.506029                0.035609           6.931378            2       True          8
4        LightGBM_BAG_L2   0.853077      79.476247  18288.997682                3.063341         341.993953            2       True          9
5   TextPredictor_BAG_L1   0.850471      39.646497  14920.726655               39.646497       14920.726655            1       Tr